# 🗺️ Planners (with FunctionCallingStepwise planner)

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.7.1-preview"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Abstractions, 1.7.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.7.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha

### 🔌 Let's add a native plugin for use by the planner

In [2]:
#!import Helpers/PrintHelper.cs
#!import Plugins/MotorPlugin.cs

kernel.Plugins.AddFromType<MotorPlugin>();

PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.


### 📋 Prepare kernel arguments

In [3]:
var goal = "You have a tree in front of the car. Avoid it.";
var kernelArguments = new KernelArguments()
{
    ["input"] = goal,
    ["commands"] = "go forward, go backward, turn left, turn right, and stop"
};

### Refine the goal

In [4]:
var commandsPlugin = kernel.CreatePluginFromPromptDirectory(System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "Plugins", "CommandsPlugin"), "CommandsPlugin");
kernel.Plugins.Add(commandsPlugin);
var semanticFunction = commandsPlugin["ExtractBasicCommands"];
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);
var refinedGoal = response.GetValue<string>();

kernelArguments["input"] = refinedGoal;

display(refinedGoal)

Here's a breakdown of the action into basic motor commands:

- go forward (until there's a need to start the avoidance maneuver)
- turn right (to avoid the tree)
- go forward (to clear the area around the tree)
- turn left (to re-align with the original direction, assuming you've passed the tree)
- go forward (to maintain the original path, if necessary)
- turn left (to get back onto the original path, if you turned off of it to avoid the tree)
- go forward (if additional distance is needed to be back on the original path)
- stop (to bring the car to the final state as required)

### 🗺️🧠 Generate FunctionCallingStepwise Plan

This planner is different from the handlebarsplanner in that it doesn't generate a plan ahead of time, and simply progresses towards its goal.

### 🏃 Let's see it run

In [5]:
using Microsoft.SemanticKernel.Planning;

#pragma warning disable SKEXP0060

var functionCallingStepwisePlannerOptions = new FunctionCallingStepwisePlannerOptions
{
    MaxIterations = 30
};

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");

PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.
🔌 CommandsPlugin
   📦 /ExtractBasicCommands: Break down complex actions into basic motor commands.
      📥 Params:
       • input: Complex action to be break down into basic motor commands. (default: '')
       • commands: The basic motor commands to choose from. (default: '')


In [6]:
#pragma warning disable 

var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)

🡲
🡶
🡲
🡵
🡲
🡵
🡲
·


The avoidance maneuver has been completed, and the car is now stopped as required. The car is back on the original path.

In [7]:
#pragma warning disable SKEXP0060

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");
var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)

🡲
🡶
🡲
🡵
🡲
🡵
🡲
·


The car has successfully completed the avoidance maneuver and has stopped, returning to the original path.